In [0]:
!pip install -U tensorflow

In [0]:
pip install patool

In [0]:
from google.colab import files
import patoolib
import io
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from shutil import copyfile

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [0]:
chk = os.listdir('.')
print(chk)

In [0]:
!rm 'test_subjects2.zip'

In [0]:
uploaded = files.upload()

In [0]:
patoolib.extract_archive("dataset.rar", outdir=".")

In [0]:
!mkdir a
!mkdir b
!mkdir c
!mkdir a/images
!mkdir b/images
!mkdir c/images

In [0]:
!rm -r dataset

In [0]:
print(os.listdir('dataset'))

In [0]:
dirs = os.listdir('dataset')
for d in dirs:
  cmn = './dataset/' + d + '/' + d
  nme =  d + '.png'
  copyfile(cmn + '_table.png', './a/images/' + nme)
  copyfile(cmn + '_mask.png', './b/images/' + nme)
  copyfile(cmn + '_mask_cell.png', './c/images/' + nme)

In [0]:
print(os.listdir('./a/images'))

In [0]:
table_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)
col_mask_datagen = ImageDataGenerator(
        rescale=1./20,
        fill_mode='constant',
        cval=0)
cell_mask_datagen = ImageDataGenerator(
        rescale=1./20,
        fill_mode='constant',
        cval=0)

In [0]:
seed=1
table_generator = table_datagen.flow_from_directory('./a', target_size=(740, 1100), class_mode=None, batch_size=10, seed=seed)
col_mask_generator = col_mask_datagen.flow_from_directory('./b', target_size=(740, 1100), class_mode=None, batch_size=10, seed=seed)
cell_mask_generator = cell_mask_datagen.flow_from_directory('./c', target_size=(740, 1100), class_mode=None, batch_size=10, seed=seed)

In [0]:
#Funkcja z internetu
def new_generator(a,b):
  while True:
    a1 = a.next()
    b1 = b.next()
    yield(a1, b1)

In [0]:
col_train_generator = new_generator(table_generator, col_mask_generator)
cell_train_generator = new_generator(table_generator, cell_mask_generator)

In [0]:
print(col_train_generator)

In [0]:
tab = table_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
tab = col_mask_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
#Sprawdzenie transpozycji
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
print(T(np.array([arr])))
#O kurwa jest dobrze :o

In [0]:
#Sprawdzenie reshapea
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
P = keras.layers.Reshape((2, 9))
print(P(T(np.array([arr]))))
#O kurwa jest dobrze :o

In [0]:
col_model = keras.Sequential([
  keras.layers.Permute((2, 1, 3), input_shape=(740, 1100, 3)), #Transpozycja obrazka.
  keras.layers.Reshape((1100, 2220)), #Pozbycie sie kolorow.
  keras.layers.Dense(740, activation='relu'), #Zmniejszenie wysokosci obrazka.
  keras.layers.Dense(74, activation='relu'), #Zmniejszenie wysokosci obrazka.
  keras.layers.Reshape((1100, 74, 1)), #Dodanie kanalu bo warstwa konwolucyjna chce.
  keras.layers.Convolution2D(filters=1, kernel_size=(101, 74), input_shape=(1100, 74, 1)), #Przejscie w poziomie.
  keras.layers.Reshape((1000, 1)),  #Wywalenie niepotrzebnego wymiaru.
  keras.layers.Convolution1D(filters=1, kernel_size=11), #Przyblizenie konca kolumn. Moze rekurencja?
  keras.layers.Reshape([990]), #Splaszczenie.
  keras.layers.Dense(1100, activation='relu'), #Lokalizacja kolumn i przywrócenie orginalnej szerokosci.
  keras.layers.Reshape((1100, 1)), #Dodanie wymiaru.
  keras.layers.Dense(2220, activation='relu'), #Odtworzenie wysokosci.
  #Zapomnialem ze jest tez puste pole dookola :( liczylem na wpisanie tej samej wartosci na calej wysokosci.
  keras.layers.Reshape((1100, 740, 3)), #Przywrócenie kanałow.
  keras.layers.Permute((2, 1, 3), input_shape=(1100, 740, 3)) #Ponowna transpozycja.
])

In [0]:
col_model.compile(loss='mse', optimizer='rmsprop')

In [0]:
his = col_model.fit_generator(
    col_train_generator,
    use_multiprocessing=True,
    steps_per_epoch=20,
    epochs=50,
    validation_data=col_train_generator,
    validation_steps=20)

In [0]:
tab = table_generator.next()[0]

In [0]:
array_to_img(tab)

In [0]:
prd = col_model.predict(np.array([tab]))
#tab
#array_to_img(col_model(tab))

In [0]:
print(prd[0])

In [0]:
array_to_img((prd[0])) #Transpozycja była jednak ok i zaczęły się pojawiać pionowe paski przy dłuższum uczeniu.
#Kończą się one mniej więcej tam gdzie tabelka. To jakiś sukces :)